In [50]:
# Import libraries
from datetime import datetime, timedelta
import numpy as np

In [51]:
# For the global variables
FILE = 'working_hours.txt'

TOTAL_MINUTES_HOUR = 60
DAILY_HOURS_TO_COVER = 8
NUM_WORKING_DAYS = 5

TOTAL_HOURS_TO_COVER = NUM_WORKING_DAYS * DAILY_HOURS_TO_COVER
MAX_MINUTES_RECOMMENDED = TOTAL_MINUTES_HOUR * DAILY_HOURS_TO_COVER
MAX_MINUTES_WEEKLY = TOTAL_MINUTES_HOUR * TOTAL_HOURS_TO_COVER
MAX_MINUTES_OVERTIME = TOTAL_MINUTES_HOUR * (DAILY_HOURS_TO_COVER + 1)

In [52]:
# Get the current time
def get_current_time():
    return str(datetime.now().hour) + ':' + str(datetime.now().minute)

# Return the total time covered in Xh and Ym
def get_hours_minutes(total_minutes):

    hours = int(total_minutes/60)
    minutes = int(total_minutes % 60)

    return str(hours) + 'h ' + str(minutes) + 'm'

# Find the difference of two timeframes
def get_duration(times):
    
    minutes_covered = 0

    # Each time gap has two outcomes: 
    # 1. 1400-1800 (240 minutes) 
    # 2. 0800- (Current time)
    for hours in times:
        hours = hours.rstrip('\n')
        
        # Split the string based on '-'
        (start_time, end_time) = hours.split('-')
        FMT = '%H:%M'

        end_time = end_time if end_time != '' else get_current_time()

        # Get the time difference
        diff = datetime.strptime(end_time, FMT) - datetime.strptime(start_time, FMT)
        minutes_covered += diff.seconds/60

    return int(minutes_covered)

def get_day_number():
    return datetime.today().weekday()

def calculate_hours():
    total_covered = 0

    days_array = []
    minutes_array = []
    finishing_time_today = ''
    
    day_number = get_day_number()

    # Load text file
    with open(FILE) as file:

        # One line = Day, Time gap #1, Time gap #2, ..., Time gap n
        for index, line in enumerate(file):
            
            line_array = line.replace('\x00','').rstrip().split(',')
            
            day = line_array[0]
            day_coverage = line_array[1:]

            minutes_covered = get_duration(day_coverage)
            
            days_array.append({
                'day': day,
                'minutes_covered': minutes_covered,
                'hours_covered': get_hours_minutes(minutes_covered),
                'coverage': day_coverage
            })
            
            if index == day_number:
                
                remaining = MAX_MINUTES_RECOMMENDED - minutes_covered
                remaining = 0 if remaining <= 0 else remaining

                finishing_time_today = datetime.now() + timedelta(minutes = remaining)

            total_covered += minutes_covered

    # Calculate overall week stats
    total_covered = total_covered if total_covered <= MAX_MINUTES_WEEKLY else MAX_MINUTES_WEEKLY

    return (total_covered, days_array, finishing_time_today)

In [53]:
# Find:
# - the total hours covered
# - the days covered so far and the breakdown of minutes per day

(total_covered, days_array, finishing_time_today) = calculate_hours()

In [54]:
# Convert the days_array to dataframe
import pandas as pd

days_df = pd.DataFrame(days_array)
print(days_df)
print('\n\n')
print('FINISHING TIME TODAY (TO COVER 8 WORKING HOURS): ', finishing_time_today.time().strftime("%H:%M:%S"))

   day  minutes_covered hours_covered                                 coverage
0  Mon              438        7h 18m  [09:12-11:12, 11:16-11:48, 12:06-16:52]
1  Tue              404        6h 44m               [08:37-11:38, 13:49-17:32]
2  Wed              139        2h 19m                                 [08:10-]
3  Thu                0         0h 0m                                       []
4  Fri                0         0h 0m                                       []



FINISHING TIME TODAY (TO COVER 8 WORKING HOURS):  16:10:02


In [55]:
# Bar chart
import plotly.express as px

fig = px.bar(days_df, 
             x='minutes_covered', y='day',
            hover_data=['coverage', 'hours_covered'],
            labels={
                'day': 'Day', 
                'minutes_covered': 'Total (Minutes)', 
                'hours_covered': 'Total (Hours)',
                'coverage': 'Day coverage'
            },
            orientation='h',
            height = 400)

fig.update_layout(
    title_text='Weekly hours calculation (Individual Days)',
    yaxis=dict(autorange="reversed"))

fig.add_shape(
        # Line Vertical
        dict(
            type="line",
            x0=MAX_MINUTES_RECOMMENDED,
            x1=MAX_MINUTES_RECOMMENDED,
            y0=-0.5,
            y1=4.5,
            line=dict(
                color="#FECB52",
                width=3
            )
))

fig.update_shapes(dict(xref='x', yref='y'))

fig.show()

In [56]:
remaining_time = MAX_MINUTES_WEEKLY - total_covered if MAX_MINUTES_WEEKLY > total_covered else 0

# Convert to dataframe
total_calculation_df = pd.DataFrame([
    {
        'category': 'covered', 
        'amount': total_covered, 
        'amount_hrs': get_hours_minutes(total_covered)
    },
    {
        'category': 'remaining', 
        'amount': remaining_time,
        'amount_hrs': get_hours_minutes(remaining_time)
    }
])

# Find the average time to cover on a daily basis to fulfill 40hours
day_number = get_day_number()

total_time_exclude_today = remaining_time + days_array[day_number]['minutes_covered']
days_remaining = NUM_WORKING_DAYS - day_number

# We first find the average of minutes needed to be covered per day
avg_mins, remaining_mins = divmod(total_time_exclude_today, (days_remaining))

# Then we add the products of the previous command
# and convert them into hours and minutes format, as our average
avg_hour, avg_mins = divmod(avg_mins + remaining_mins, TOTAL_MINUTES_HOUR)

print("AVERAGE TIME TO COVER DAILY: {}h {}m".format(avg_hour, avg_mins))

AVERAGE TIME TO COVER DAILY: 8h 40m


In [57]:
# Pie chart
import plotly.graph_objects as go

fig = go.Figure(go.Pie(
    name="",
    values = total_calculation_df['amount'],
    labels = total_calculation_df['category'],
    customdata = total_calculation_df['amount_hrs'],
    hovertemplate = "Category: %{label} <br>Total(minutes): %{value} </br>Total(hours): %{customdata}"

))

fig.update_layout(title_text='Overall weekly hours calculation: remaining vs covered')

fig.show()